# Classification d'images de fleurs

Sur ce notebook on va parcourir toutes les étapes pour implémenter un réseau convolutif qui fait de la classification d'images.

## Imports de modules

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import Sequential, layers

SEED = 123
plt.style.available
plt.style.use("seaborn-talk")
plt.style.use("seaborn-whitegrid")

## Les données
Pour entraîner notre modèle, on va avoir besoin d'images de fleurs étiquetés. On s'utilisera de la base d'exemple [tf_images](https://www.tensorflow.org/datasets/catalog/tf_flowers).

On peut explorer le dataset à l'aide de l'outil suivant:
- [Exploration du dataset avec KnowYourData](https://knowyourdata-tfds.withgoogle.com/#tab=STATS&dataset=tf_flowers)

### Téléchargement des données

De toute façon, on devra télécharger les donnés sur notre espace de travail pour dévélopper le modèle.

In [ ]:
import pathlib

dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
data_dir = tf.keras.utils.get_file("flower_photos", origin=dataset_url, untar=True)
data_dir = pathlib.Path(data_dir)

Le dataset contient 3670 photos de fleurs, organisees en 5 dossiers:

```
flower_photo/
  daisy/
  dandelion/
  roses/
  sunflowers/
  tulips/
```

In [ ]:
image_count = len(list(data_dir.glob("*/*.jpg")))
print(image_count)

### Génération des datasets train et validation

On gardera 20% des données pour la validation.

In [ ]:
taille_pixels = 180  # @param {type: "number"}
batch_size = 32  # @param {type: "number"}
img_height, img_width = taille_pixels, taille_pixels
img_size = (img_height, img_width)

In [ ]:
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="training",
    seed=SEED,
    image_size=(img_height, img_width),
    batch_size=batch_size,
)

In [ ]:
val_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="validation",
    seed=SEED,
    image_size=(img_height, img_width),
    batch_size=batch_size,
)

In [ ]:
class_names = train_dataset.class_names
num_classes = len(class_names)
print(class_names)

#### L'objet dataset
Les objets retournés fonctionnent comme un `iterator` Python. C'est à dire qu'on peut l'utiliser directement dans une boucle for comme ici:

In [ ]:
for element in train_dataset:
    print("type:", type(element), "taille:", len(element))
    break

In [ ]:
type(train_dataset)

Les éléments qu'on accède à chaque fois dans la boucle sont des tuples avec deux éléments. Le premier est une image, et le deuxième son étiquette. On peut les séparer dans la boucle ainsi:

In [ ]:
for image, label in train_dataset:
    print(image.shape)
    print(label.shape)
    break

### Exploration rapide

#### Regardons quelques images

Ici un bout de code pour vous afficher quelques images de la base.

In [ ]:
# @title Quelques images du train set { display-mode: "form" }
max_images = "9"  # @param ["4", "9", "16"]
max_images = int(max_images)
count = 0
gs = grid_size = int(np.sqrt(max_images))
fig, axs = plt.subplots(gs, gs, sharex=True, sharey=True, figsize=(gs * 3, gs * 3))
axs = axs.flatten()
for image, label in train_dataset:
    ax = axs[count]
    ax.imshow(image[0].numpy().astype(np.uint8))
    ax.axis("off")
    ax.set_title(str(label[0].numpy()))
    count += 1
    if count >= max_images:
        break

In [ ]:
# @title Quelques images de valdation { display-mode: "form" }
max_images = "4"  # @param ["4", "9", "16"]
max_images = int(max_images)
count = 0
gs = grid_size = int(np.sqrt(max_images))
fig, axs = plt.subplots(gs, gs, sharex=True, sharey=True, figsize=(gs * 3, gs * 3))
axs = axs.flatten()
for image, label in val_dataset:
    ax = axs[count]
    ax.imshow(image[0].numpy().astype(np.uint8))
    ax.axis("off")
    ax.set_title(str(label[0].numpy()))
    count += 1
    if count >= max_images:
        break

#### Histogramme des étiquettes

On accumule les étiquettes dans une liste et puis on la transforme en array numpy:

In [ ]:
label_values = np.arange(num_classes, dtype=np.int32)

In [ ]:
labels = []
for image, label in train_dataset:
    labels.append(label.numpy())

train_labels = np.concatenate(labels)
print("Shape de l'array labels: ", train_labels.shape)

On fait le même pour l'ensemble de validation pour vérifier si la distribution est semblable à celle de l'entraînement.

In [ ]:
labels = []
for image, label in val_dataset:
    labels.append(label.numpy())
val_labels = np.concatenate(labels)
print("Shape de l'array labels: ", val_labels.shape)

In [ ]:
# @title Distribution des classes { display-mode: "form" }
plt.hist(
    train_labels,
    bins=num_classes,
    rwidth=0.8,
    align="left",
    label="train",
    density=True,
)
plt.hist(
    val_labels,
    bins=num_classes,
    rwidth=0.8,
    align="mid",
    label="valid",
    density=True,
)
plt.xticks(label_values, label_values)
plt.legend()
plt.title("distribution des categories sur les partitions")

#### Les valeurs des pixels

Remarquez que les valeurs des pixels RGB varient entre 0 et 255. 
Pour les réseaux de neuronnes, il vaut mieux garder les valeurs d'entrée petites, come par exemple entre [0,1]. Il nous faudra donc diviser la valeur des pixels par 255.

In [ ]:
# @title Affichage des 3 canaux individuellement { display-mode: "form" }
for image, label in train_dataset:
    img = image[0].numpy()
    cmaps = ["Reds", "Greens", "Blues"]
    fig, axs = plt.subplots(1, 4, figsize=(15, 3))
    axs[3].imshow(img.astype("uint8"))
    ax.set_title("Image RBG")
    for ch in range(3):
        ax = axs[ch]
        imgplt = ax.imshow(img[..., ch], cmap=cmaps[ch])
        ax.set_title("Canal " + cmaps[ch][0])
        fig.colorbar(imgplt, ax=ax)
    break
plt.tight_layout()

In [ ]:
# @title def de la fonction image_hist { display-mode: "form" }
def image_hist(img):
    fig, axs = plt.subplots(1, 4, figsize=(12, 4))
    ch_name = ["rouge", "vert", "bleu"]
    color = "rgb"
    for i in range(3):
        channel = img[..., i]
        ax = axs[i]
        ax.hist(
            channel.flatten(),
            orientation="horizontal",
            color=color[i],
            range=(0, 256),
        )
        ax.set_title(ch_name[i])
        ax.xaxis.set_visible(False)
    fig.suptitle("histogramme des pixels d'une image")
    ax = axs[-1]
    ax.imshow(img.astype("uint8"))
    ax.axis("off")
    print("pixels: max=", img.max(), "min=", img.min())

In [ ]:
for image, label in train_dataset:
    img = image[0].numpy()
    image_hist(img)
    break

## Le modèle

### Création du modèle `Sequential`

In [ ]:
num_classes = len(train_dataset.class_names)

model = Sequential(
    [
        layers.InputLayer(input_shape=(img_height, img_width, 3), name="input"),
        layers.experimental.preprocessing.Rescaling(1.0 / 255),
        layers.Conv2D(16, 3, padding="same", activation="relu", name="conv1"),
        layers.MaxPooling2D(name="pool1"),
        layers.Conv2D(32, 3, padding="same", activation="relu", name="conv2"),
        layers.MaxPooling2D(name="pool2"),
        layers.Conv2D(64, 3, padding="same", activation="relu", name="conv3"),
        layers.MaxPooling2D(name="pool3"),
        layers.Flatten(name="pool3_flat"),
        layers.Dense(128, activation="relu", name="dense4"),
        layers.Dense(num_classes, name="output"),
    ],
    name="cnn_flowers",
)
model.save_weights("init.h5")

In [ ]:
model.summary()

### Visualisation

In [ ]:
keras.utils.plot_model(model, show_layer_names=False, rankdir="LR")

In [ ]:
keras.utils.plot_model(model, show_layer_names=False, show_shapes=True, rankdir="TB")

In [ ]:
try:
    import visualkeras as vk
except ModuleNotFoundError:
    !pip install visualkeras;
    import visualkeras as vk

vk.layered_view(model, legend=True, scale_xy=1)

## L'entrainement

### Lecture rapide des batchs
Pour que la lecture des `batch`s de données soit plus efficace, `tf.Dataset` nous fournit quelques méthodes adaptés. Elles nous transforment l'objet originel dans un autre avec une fonctionnalité en plus.

Ici on utilisera:
- cache: garder en mémoire les images une fois chargées
- prefetch: pre-charger un certain nombre de batches en mémoire


In [ ]:
train_ds = train.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds = val.cache().prefetch(buffer_size=tf.data.AUTOTUNE)

### `compile` et `fit`

In [ ]:
model.compile(
    optimizer="adam",
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)

In [ ]:
# @title Training loop
reinit = True  # @param {type: "boolean"}
epochs = 10  # @param {type: "number"}
learning_rate = 0.001  # @param {type: "number"}

model.optimizer.learning_rate = learning_rate

if reinit:
    model.load_weights("init.h5")

out = model.fit(train, validation_data=val_ds, epochs=epochs)

### Visualiser les courbes d'apprentissage

In [ ]:
# @title Def de la fonction learning_curves { display-mode: "form" }
def learning_curves(out, **kwargs):
    metrics = out.history.keys()
    # collecter les noms des metriques enregistrés sur history
    metrics = list(filter(lambda m: not m.startswith("val"), metrics))
    num_metrics = len(metrics)
    # creer un gid de subplots approprié
    fig, axs = plt.subplots(
        num_metrics,
        1,
        figsize=(9, 4 * num_metrics),
    )
    # afficher chaque type de métrique dans son plot
    for i, metric in enumerate(metrics):
        loss = out.history[metric]
        val_loss = out.history["val_" + metric]
        epochs_range = range(1, len(loss) + 1)
        ax = axs[i]
        ax.plot(
            epochs_range,
            loss,
            marker=".",
            linestyle="dashed",
            label="Train " + metric,
            **kwargs
        )
        ax.plot(
            epochs_range,
            val_loss,
            marker=".",
            linestyle="dashed",
            label="Valid " + metric,
            **kwargs
        )
        ax.legend()
        ax.set_title(metric)
    ax.set_xlabel("epochs")
    fig.suptitle("courbes d'apprentisage x époques", fontsize="x-large")

In [ ]:
learning_curves(out)

## Diagnostique : sur-apprentissage

On peut remarquer sur les courbes d'apprentissage:
- un erreur d'entraînement quand même très bas;
- un grand écart entre les métriques `train` et `val`, ce qui indique un grand erreur de généralisation;

Ses symptomes indiquent que le modèle souffre de sur-apprentissage.

## Solutions pour un modèle 2.0
Pour l'ameliorer, on peut prendre certaines mesures comme:

- augmenter la quantité de données
- réduire la complexité du modèle
- introduire de la régularisation

On va ici utiliser deux strategies:
- le _data augmentation_: une augmentation artificielle des données
- le _dropout_: une technique de régularisation

### Data augmentation

In [ ]:
data_augmentation = keras.Sequential(
    [
        layers.InputLayer(input_shape=(img_height, img_width, 3)),
        layers.experimental.preprocessing.RandomFlip(mode="horizontal"),
        layers.experimental.preprocessing.RandomRotation(factor=0.1),
        layers.experimental.preprocessing.RandomZoom(height_factor=0.1),
    ]
)

In [ ]:
plt.figure(figsize=(10, 10))
for images, _ in train_ds.take(1):
    for i in range(9):
        augmented_images = data_augmentation(images)
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(augmented_images[0].numpy().astype("uint8"))
        plt.axis("off")

### Dropout

In [ ]:
model2 = Sequential(
    [
        data_augmentation,
        layers.experimental.preprocessing.Rescaling(1.0 / 255),
        layers.Conv2D(16, 3, padding="same", activation="relu", name="conv1"),
        layers.MaxPooling2D(name="pool1"),
        layers.Conv2D(32, 3, padding="same", activation="relu", name="conv2"),
        layers.MaxPooling2D(name="pool2"),
        layers.Conv2D(64, 3, padding="same", activation="relu", name="conv3"),
        layers.MaxPooling2D(name="pool3"),
        layers.Dropout(0.2),
        layers.Flatten(name="pool3_flat"),
        layers.Dense(128, activation="relu", name="dense4"),
        layers.Dense(num_classes, name="output"),
    ],
    name="cnn_flowers2",
)

Suite à l'inclusion des couches d'augmentation, on a des changements dans le backend qui font que l'on doive faire un appel a la méthode `build` ou appeler le réseau sur un batch d'échantillons pour faire initialiser ses poids.

In [ ]:
model2.build(input_shape=(batch_size, img_height, img_width, 3))
# ou 
# model2(rng.rand(batch_size, img_height, img_width, 3))

Maintenant on peut les sauvegarder sous un nouveau nom à fin de ne pas écraser celui du précédent réseau.

In [ ]:
model2.save_weights("init2.0.h5")

## Entraînement 2.0

### `compile` et `fit`

In [ ]:
model2.compile(
    optimizer="adam",
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)

In [ ]:
# @title Training loop
reinit = True  # @param {type: "boolean"}
epochs = 15  # @param {type: "number"}
learning_rate = 0.001  # @param {type: "number"}

model2.optimizer.learning_rate = learning_rate

if reinit:
    model2.load_weights("init2.0.h5")

out2 = model2.fit(train_ds, validation_data=val_ds, epochs=epochs)

### Courbes d'apprentissage

In [ ]:
learning_curves(out2)

**Optionnel**: Exécutez le code suivant si vous voulez sauvegarder votre `model2` dans un fichier:

In [ ]:
# model2.save('model2.0.h5')

## Test final
On essaiera de faire des prédictions sur d'autres images de fleurs:

In [ ]:
urls = [
    "https://storage.googleapis.com/download.tensorflow.org/example_images/592px-Red_sunflower.jpg",
    "https://d2j6dbq0eux0bg.cloudfront.net/images/9350281/982837658.jpg",
    "https://www.roses-andre-eve.com/2811-large_default/aspirin-rose-taniripsa-.jpg",
    "https://cdn11.bigcommerce.com/s-f74ff/images/stencil/1280x1280/products/9531/29981/wetland-plants-dandelion-plant-dandelion__86787.1600975395.jpg?c=2",
    "https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSzXukWgU6XFSKQWMLkE-ZVnuTCbQESSWMYvQ&usqp=CAU",
    "https://www.rhs.org.uk/getmedia/207b5a2a-2332-4863-8990-070175545f92/Tulip-Fusilier-C-Dorling-Kindersley.jpg?width=940&height=627&ext=.jpg",
    "https://ak.picdn.net/shutterstock/videos/3242563/thumb/1.jpg",
    "https://www.josephotos.com/wp-content/uploads/2014/03/DSC_7255_daisies.jpg",
    "https://upload.wikimedia.org/wikipedia/commons/3/3b/Wooden_Shoe_Tulip_Farm_2011_-_Oregon_%285649088598%29.jpg",
]
names = [
    "Sunflower",
    "Rainbow-rose",
    "Rose-claire",
    "Dandelion",
    "Close-Dandelion",
    "Tulipes-ouvertes",
    "daisy-field",
    "daisy-top",
    "tulip-field",
]

**Optionnel**: Exécutez le code suivant si vous avez sauvegardé un modèle et que vous voulez le charger pour l'utiliser:

In [ ]:
# model2 = keras.models.load_model('model2.0.h5')

Ici le code télécharge les images choisis:

In [ ]:
img_paths = []
for url, name in zip(urls, names):
    img_path = tf.keras.utils.get_file(name, origin=url)
    img_paths.append(img_path)

Ensuite on va remettre ces images sous un format que le réseau va reconnaître (en ce qui concerne sa taille en pixels et sa représentation en forme d'array 3D/4D):

In [ ]:
i = 0
for name, img_path in zip(names, img_paths):
    print("\n{}".format(name))
    img = keras.preprocessing.image.load_img(
        img_path, target_size=(img_height, img_width)
    )
    plt.style.use("default")
    plt.subplot(3, 3, i + 1)
    plt.imshow(img)
    plt.title(name)
    plt.axis("off")
    i += 1

    img_array = keras.preprocessing.image.img_to_array(img)

    img_array = tf.expand_dims(img_array, 0)  # Create a batch

    predictions = model.predict(img_array)
    score = tf.nn.softmax(predictions[0])

    print(
        "Image de la classe {} avec  {:.2f}\% de confiance.".format(
            class_names[np.argmax(score)], 100 * np.max(score)
        )
    )